In [12]:
import os
import sys
import pandas
import numpy, scipy, sklearn

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [13]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [14]:
file_name = "/Users/simondi/PHD/data/data/target_infect_x/query_data/cells_sample_10_normalized_cut_100.tsv"

In [15]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-4-b848b1fa3d33>:2 

In [17]:
df = spark.read.csv(path=file_name, sep="\t", header='true')

In [36]:
data = df.toPandas()
data = data.drop(columns="cells.children_invasomes_count")

In [37]:
fc = list(filter(lambda x: x.startswith("cells"), data.columns))
data = data[fc]

In [38]:
data.to_csv("/Users/simondi/PHD/data/data/target_infect_x/query_data/cells_sample_10_normalized_cut_100_X.tsv", sep="\t", header=False, index=False)

In [39]:
data = data.as_matrix()
data = data.astype(numpy.float64)

In [52]:
# factor analysis
means = numpy.mean(data, axis=0)
std = numpy.nanstd(data, axis=0)
X = (data -means)

In [23]:
from sklearn import decomposition

In [24]:
fa = sklearn.decomposition.FactorAnalysis(2, svd_method="lapack")
Fact = fa.fit(X)

In [45]:
Fact.transform(X)[:5, :]

array([[-0.12770892, -0.9039069 ],
       [-0.39663874, -1.03977064],
       [-0.86838462, -0.01130043],
       [-0.83457665, -0.27794946],
       [-1.23926041, -0.24232402]])

In [26]:
def my_svd(X, n_components):
    _ , s, V = scipy.linalg.svd(X, full_matrices=False)
    squared_norm = numpy.dot(s[n_components:], s[n_components:])
    return (s[:n_components], V[:n_components], squared_norm)

In [107]:
iter = 0
SMALL = 1e-12
ll = old_ll = -numpy.inf
n_samples, n_features = X.shape
psi = numpy.ones(n_features, dtype=X.dtype)
var = numpy.var(X, axis=0)
n_components = 2
nsqrt = numpy.sqrt(n_samples)
llconst = n_features * numpy.log(2. * numpy.pi) + n_components
print(nsqrt)

10.0


In [108]:
for i in range(5):
    print(i)
    sqrt_psi = numpy.sqrt(psi) + SMALL
    s, V, unexp_var = my_svd(X / (sqrt_psi * nsqrt), 2)
    s **= 2
    W = numpy.sqrt(numpy.maximum(s - 1., 0.))[:, numpy.newaxis] * V
    W *= sqrt_psi    
    ll = llconst + numpy.sum(numpy.log(s))
    ll += unexp_var + numpy.sum(numpy.log(psi))
    ll *= -n_samples / 2.    
    psi = numpy.maximum(var - numpy.sum(W ** 2, axis=0), SMALL)
    if numpy.abs(ll - old_ll) < 0.001:
        break
    old_ll = ll

0
1
2
3
4


In [109]:
Ih = numpy.eye(len(W))
X_transformed = X
Wpsi = W / psi
cov_z = scipy.linalg.inv(Ih + numpy.dot(Wpsi, W.T))
tmp = numpy.dot(X_transformed, Wpsi.T)
X_transformed = numpy.dot(tmp, cov_z)
X_transformed[:5, :]

array([[-0.08085689, -0.88416287],
       [-0.32554325, -0.9936295 ],
       [-0.85995564,  0.00164118],
       [-0.80788002, -0.29778315],
       [-1.21530389, -0.14961542]])